In [18]:
import os
import pandas as pd
import geopandas as gpd
import json
from ipyleaflet import Map, Choropleth, Polyline #, GeoData, basemaps, LayersControl,
from branca.colormap import linear

In [19]:
kmeans_results = pd.read_csv(os.path.join('..', 'trampact', 'data', 'pca_kmeans_results', 'census_2006_nice_tx_kmeans=5.csv'))

In [20]:
kmeans_results['iris_id'] = kmeans_results.iris_id.map(lambda x: str(0) + str(x))

In [21]:
iris_clusters = dict(
    zip(
        kmeans_results['iris_id'].tolist(), kmeans_results['pca_labels'].tolist()
        )
    )

In [22]:
# Load coordinates of tramway line
line_t1 = pd.read_csv('gs://trampact_storage/data/coord_T1.csv')
line_t1.drop(columns='Unnamed: 0',inplace=True)
line_t2 = pd.read_csv('gs://trampact_storage/data/lignt_t2_coord.csv')

points_t1 =[]
for i in range(len(line_t1)):
    t=[line_t1.loc[i,'y'],line_t1.loc[i,'x']]
    points_t1.append(t)
    

points_t2 =[]
for i in range(len(line_t2)):
    t=[line_t2.loc[i,'lat'],line_t2.loc[i,'lon']]
    points_t2.append(t)# Load geograpy data

zipfile = "../raw_data/iris_geo/iris-geo-2018-frtot.zip"

In [23]:
gdf = gpd.read_file(zipfile) # geopandas.dataframe

# gdf.drop(columns=['INSEE_COM','NOM_COM','IRIS','NOM_IRIS'],inplace=True)
gdf.drop(columns=['INSEE_COM','IRIS','NOM_IRIS'],inplace=True)

# Filter data for Nice region
gdf = gdf[gdf['NOM_COM'] == 'Nice'].reset_index().drop(columns=['index'])

In [24]:
# Convert geopandas.dataframe to json format in the form of a string
gdf_json=gdf.to_json()

# Convert json string object to json dictionary
dict_json = json.loads(gdf_json)

# Assign id the value of iris_id, since matching of geo and feature dictionary is made on id in dict_json
for feature in dict_json['features']:
    feature['id']=feature['properties']['CODE_IRIS']# Load csv data
feature_df_csv = pd.read_csv('../trampact/data/BD_GENT_2006.csv')

# Correction for zero in the beginning of iris_id
feature_df_csv.iris_id=feature_df_csv.iris_id.map(lambda x: str(0)+str(x))

# transform dataframe into dictionary with iris_id as keys and the feature of interest as value
# ftr = dict(zip(feature_df_csv['iris_id'].tolist(), feature_df_csv['actifs_2006'].tolist()))

In [25]:
center=(43.723348,7.285484)
zoom = 12
m = Map(center = center, zoom = zoom)

'''geo_data = GeoData(geo_dataframe = join_evan_df,
                   style={'color': 'black', 'fillColor': '#3366CC
', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'iris_id')
                   '''
line = Polyline(
    locations=points_t1,
    style={'color': 'black', 'fillColor': '#3366CC', 'opacity':0.5, 'weight':9, 'dashArray':'2', 'fillOpacity':0.6},
    color="green" ,
    fill=False)

line2 = Polyline(
    locations=points_t2,
    style={'color': 'black', 'fillColor': '#3366CC', 'opacity':0.5, 'weight':9, 'dashArray':'2', 'fillOpacity':0.6},
    color="blue" ,
    fill=False)

layer = Choropleth(
    geo_data=dict_json,
    choro_data=iris_clusters,
    colormap=linear.YlOrRd_04,
    border_color='black',
    style={'fillOpacity': 0.7, 'dashArray': '5, 5'})

m.add_layer(layer)
m.add_layer(line)
# m.add_layer(line2)

#m.add_layer(geo_data)
#m.add_control(LayersControl())

In [26]:
m

Map(center=[43.723348, 7.285484], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …